In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import random
import time
%matplotlib inline
import matplotlib.animation as animation
import csv
# matplotlib.style.use('seaborn')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn import manifold
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import pairwise_distances
# from wordcloud import WordCloud
from sklearn.feature_extraction import text

from mpl_toolkits.mplot3d import Axes3D

In [2]:
blog_data=pd.read_csv("blogtext.csv")

In [3]:
blog_data.drop_duplicates(subset="text",inplace=True)
blog_data.date = pd.to_datetime(blog_data.date,format="%d,%B,%Y", errors='coerce')#pd.to_datetime(blog_data.date,errors="coerce",infer_datetime_format=True)
print(blog_data.shape)
blog_data.sample(20)

(611652, 7)


,id,gender,age,topic,sign,date,text
377090,2297959,female,15,Student,Pisces,2004-08-02,i want a boyfriend
17699,3723457,female,24,indUnk,Sagittarius,2004-07-08,Swimmin' in the pool&nbsp; urlLink ...
42178,888601,female,24,indUnk,Libra,2004-03-18,I was trying to watch my Charles Manson...
45128,2916583,female,17,Student,Gemini,2004-08-08,"Yeah, yeah, I know.&nbsp; It..."
165067,576311,female,34,indUnk,Capricorn,2004-08-02,the wedding we played at last saturday ...
205047,1266843,female,24,Communications-Media,Pisces,2004-03-18,"2 am: I got out of work around 11, pret..."
136605,1015252,female,23,indUnk,Pisces,2004-07-03,for heavens sake. my machines attack th...
304634,2142241,female,16,indUnk,Libra,2004-02-01,"dude, i wanna go to the mall..."
568072,3411223,female,34,Education,Pisces,2004-06-23,"Masturbation, what is the taboo? Tonig..."
518312,3655275,female,33,Science,Sagittarius,2004-07-12,It is a long held belief of mine that one o...


In [4]:
# ####################################################################################################
# #####                            Sampling for code develpoment 
# #####                            Remove after validating code
# ####################################################################################################

# blog_data=blog_data.sample(frac=.1)#.02

# print("******WARNING****** \n DATASET SAMPLED!!!!!!")
# print(blog_data.shape)

# Word Embeddings

In [5]:
adit_stpwrds=["urllink","nbsp","ve","ll"]
stp_wrds = text.ENGLISH_STOP_WORDS.union(adit_stpwrds)

In [ ]:
tfidf_transformer = TfidfVectorizer(stop_words=stp_wrds,max_features=1000 )
n_grams_tfidf = tfidf_transformer.fit_transform(blog_data["text"])
n_grams_tfidf.shape

In [ ]:
# plt.figure(figsize=(16,16))
# plt.spy(n_grams_tfidf,markersize=.5, marker="o",aspect='auto')
# plt.title("TF-IDF matrix")

In [ ]:
# top_n=150
# txt_info= n_grams_tfidf.toarray().sum(axis=0)
# indices = np.argsort(txt_info)[::-1]
# features =tfidf_transformer.get_feature_names()
# top_features = np.array([[features[i],float(txt_info[i])] for i in indices[:top_n]])
# pd.DataFrame({"word":top_features[:,0],"info":top_features[:,1]})[:50]

In [ ]:
# plt.figure(figsize=(16,9))
# plt.plot(txt_info[indices])#[indices[:top_n]]
# plt.title("Term info sorted")

In [ ]:
# n_grams_count_top=n_grams_tfidf.toarray()[:,indices[:top_n]]

In [ ]:
# plt.figure(figsize=(16,16))
# plt.spy(n_grams_count_top,markersize=.5,aspect='auto', marker="o")
# plt.title("Top term info matrix")

In [ ]:
# text_ward_full=hierarchy.linkage(n_grams_count_top, 'ward')#(n_grams_tfidf.toarray(),"ward")#
# text_ward_full=hierarchy.linkage(n_grams_tfidf.toarray(),"ward")

In [ ]:
# plt.figure(figsize=(15,35))
# dn_tf_full = hierarchy.dendrogram(text_ward_full,orientation="left")
# plt.title('Ward Link Dendrogram from top embedding')
# plt.show()

In [ ]:
df_out=pd.DataFrame(blog_data["id"]).join(pd.SparseDataFrame(n_grams_tfidf,
                                         columns=tfidf_transformer.get_feature_names(),
                                         index=list(blog_data.index)))#.to_csv("tfidf_pd_texts.csv")

In [ ]:
usr_txt_ftr=df_out.groupby(['id'], sort=False).sum()
usr_txt_ftr.to_csv("usr_txt_ftr.csv")

In [ ]:
# (usr_txt_ftr/max(usr_txt_ftr.max())).to_csv("usr_txt_ftr.csv")
usr_txt_ftr.to_csv("usr_txt_ftr.csv")